##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Serving을 사용한 TensorFlow 모델 훈련 및 제공

**경고: 이 노트북은 Google Colab에서만 실행되도록 설계되었습니다**. 시스템에 패키지를 설치하고 루트 권한이 필요합니다. 로컬 Jupyter 노트북에서 실행하려면 주의해서 진행하세요.

참고: 이 예제는 Jupyter 스타일 노트북에서 바로 실행할 수 있으며 설정이 필요하지 않습니다. "Google Colab에서 실행"을 클릭하세요.

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<tr>
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/serving/rest_simple"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/serving/rest_simple.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/tfx/blob/master/docs/tutorials/serving/rest_simple.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소그 보기</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/serving/rest_simple.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운론드하기</a></td>
</tr>
</table></div>

이 가이드는 신경망 모델을 훈련하여 [운동화 및 셔츠와 같은 의류 이미지](https://github.com/zalandoresearch/fashion-mnist)를 분류하고 훈련된 모델을 저장한 다음 [TensorFlow Serving](https://www.tensorflow.org/serving/)을 사용하여 제공합니다. TensorFlow의 모델링 및 훈련이 아니라 TensorFlow Serving에 초점을 맞춥니다. 따라서 모델링 및 훈련에 초점을 맞춘 전체 예제는 [기본 분류 예제](https://github.com/tensorflow/docs/blob/master/site/en/r1/tutorials/keras/basic_classification.ipynb)를 참조하세요.

이 가이드는 TensorFlow에서 모델을 빌드하고 훈련하는 데 상위 수준 API인 [tf.keras](https://github.com/tensorflow/docs/blob/master/site/en/r1/guide/keras.ipynb)를 사용합니다.

In [ ]:
import sys

# Confirm that we're using Python 3
assert sys.version_info.major == 3, 'Oops, not running Python 3. Use Runtime > Change runtime type'

In [ ]:
# TensorFlow and tf.keras
print("Installing dependencies for Colab environment")
!pip install -Uq grpcio==1.26.0

import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

print('TensorFlow version: {}'.format(tf.__version__))

## 모델 생성하기

### Fashion MNIST 데이터세트 가져오기

이 가이드는 10개 범주의 70,000개 회색조 이미지를 포함하는 [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) 데이터세트를 사용합니다. 이미지는 다음과 같이 저해상도(28x28 픽셀)의 개별 의류 품목을 보여줍니다.

<table>
  <tr><td>     <img src="https://tensorflow.org/images/fashion-mnist-sprite.png" alt="Fashion MNIST sprite">   </td></tr>
  <tr><td align="center">     <b>그림 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST 샘플</a>(Zalando, MIT License).<br>
</td></tr>
</table>

Fashion MNIST는 컴퓨터 비전용 머신러닝 프로그램의 "Hello, World"로 종종 사용되는 고전적인 [MNIST](http://yann.lecun.com/exdb/mnist/) 데이터세트를 대체하기 위한 것입니다. TensorFlow에서 직접 Fashion MNIST에 액세스하고 데이터를 가져와서 로드할 수 있습니다.

참고: 실제로는 이미지이지만, 바이너리 이미지 객체가 아닌 NumPy 배열로 로드됩니다.

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# scale the values to 0.0 to 1.0
train_images = train_images / 255.0
test_images = test_images / 255.0

# reshape for feeding into the model
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print('\ntrain_images.shape: {}, of {}'.format(train_images.shape, train_images.dtype))
print('test_images.shape: {}, of {}'.format(test_images.shape, test_images.dtype))

### 모델 훈련 및 평가하기

모델링 부분에 초점을 맞추지 않았으므로 가능한 가장 간단한 CNN을 사용하겠습니다.

In [ ]:
model = keras.Sequential([
  keras.layers.Conv2D(input_shape=(28,28,1), filters=8, kernel_size=3, 
                      strides=2, activation='relu', name='Conv1'),
  keras.layers.Flatten(),
  keras.layers.Dense(10, name='Dense')
])
model.summary()

testing = False
epochs = 5

model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])
model.fit(train_images, train_labels, epochs=epochs)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print('\nTest accuracy: {}'.format(test_acc))

## 모델 저장하기

훈련된 모델을 TensorFlow Serving으로 로드하려면 먼저 [SavedModel](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/saved_model) 형식으로 저장해야 합니다. 이렇게 하면 잘 정의된 디렉토리 계층 구조에 protobuf 파일이 생성되고 버전 번호가 포함됩니다. [TensorFlow Serving](https://www.tensorflow.org/tfx/guide/serving)을 사용하면 추론 요청을 할 때 사용할 모델 버전 또는 '서비스 가능(servable)'을 선택할 수 있습니다.  각 버전은 지정된 경로 아래의 다른 하위 디렉토리로 내보내집니다.

In [ ]:
# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors,
# and stored with the default serving key
import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

## 저장된 모델 검사하기

명령줄 유틸리티 `saved_model_cli`를 사용하여 SavedModel에서 [MetaGraphDefs](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/MetaGraphDef)(모델) 및 [SignatureDefs](../signature_defs)(호출할 수 있는 메서드)를 살펴보겠습니다. TensorFlow 가이드에서 [SavedModel CLI 토론](https://github.com/tensorflow/docs/blob/master/site/en/r1/guide/saved_model.md#cli-to-inspect-and-execute-savedmodel)을 참조하세요.

In [ ]:
!saved_model_cli show --dir {export_path} --all

모델에 대해 많은 것을 알 수 있습니다! 이 경우, 방금 모델을 훈련했으므로 이미 입력과 출력을 알고 있지만, 그렇지 않은 경우에는 중요한 정보를 알 수 있습니다. 예를 들어, 회색조 이미지 데이터라는 사실과 같이 모든 것을 알려주지는 않지만, 시작으로는 좋습니다.

## TensorFlow Serving으로 모델 제공하기

**경고: Google Colab에서 실행하지 않는 경우,** 다음 셀을 실행하면 루트 액세스 권한이 있는 시스템에 패키지가 설치됩니다. 로컬 Jupyter 노트북에서 실행하려면 주의해서 진행하세요.

### TensorFlow Serving 배포 URI를 패키지 소스로 추가합니다.

We're preparing to install TensorFlow Serving using [Aptitude](https://wiki.debian.org/Aptitude) since this Colab runs in a Debian environment.  We'll add the `tensorflow-model-server` package to the list of packages that Aptitude knows about.  Note that we're running as root.

참고: 다음 예는 기본적으로 TensorFlow Serving을 실행하지만, TensorFlow Serving을 사용하여 시작하는 가장 쉬운 방법 중 하나인 [Docker 컨테이너에서 실행할 수도 있습니다](https://www.tensorflow.org/tfx/serving/docker).

In [ ]:
import sys
# We need sudo prefix if not on a Google Colab.
if 'google.colab' not in sys.modules:
  SUDO_IF_NEEDED = 'sudo'
else:
  SUDO_IF_NEEDED = ''

In [ ]:
# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | {SUDO_IF_NEEDED} tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | {SUDO_IF_NEEDED} apt-key add -
!{SUDO_IF_NEEDED} apt update

### TensorFlow Serving을 설치합니다.

명령줄 하나가 필요한 것의 전부입니다!

In [ ]:
!{SUDO_IF_NEEDED} apt-get install tensorflow-model-server

### TensorFlow Serving 실행을 시작합니다.

여기에서 TensorFlow Serving 실행을 시작하고 모델을 로드합니다. 모델을 로드한 후 REST를 사용하여 추론 요청을 시작할 수 있습니다. 몇 가지 중요한 매개변수가 있습니다.

- `rest_api_port`: REST 요청에 사용할 포트입니다.
- `model_name`: REST 요청의 URL에서 사용할 <br> 것이며, 무엇이든 될 수 있습니다.
- `model_base_path`: 모델을 저장한 디렉토리의 경로입니다.


In [ ]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=fashion_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1


In [ ]:
!tail server.log

## TensorFlow Serving에서 모델에 요청하기

먼저, 테스트 데이터에서 임의의 예제를 살펴보겠습니다.

In [ ]:
def show(idx, title):
  plt.figure()
  plt.imshow(test_images[idx].reshape(28,28))
  plt.axis('off')
  plt.title('\n\n{}'.format(title), fontdict={'size': 16})

import random
rando = random.randint(0,len(test_images)-1)
show(rando, 'An Example Image: {}'.format(class_names[test_labels[rando]]))

흥미로워 보입니다. 여러분이 인식하기 얼마나 어렵습니까? 이제 3가지 추론 요청 배치를 위해 JSON 객체를 만들고 모델이 얼마나 잘 인식하는지 살펴보겠습니다.

In [ ]:
import json
data = json.dumps({"signature_name": "serving_default", "instances": test_images[0:3].tolist()})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

### REST 요청 만들기

#### 최신 버전의 servable

서버의 REST 엔드 포인트에 POST로 예측 요청을 보내고 3가지 예제를 전달합니다. 특정 버전을 지정하지 않음으로써 서버에 최신 버전의 servable을 요청할 것입니다.

In [ ]:
# docs_infra: no_execute
!pip install -q requests

import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/fashion_model:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

show(0, 'The model thought this was a {} (class {}), and it was actually a {} (class {})'.format(
  class_names[np.argmax(predictions[0])], np.argmax(predictions[0]), class_names[test_labels[0]], test_labels[0]))

#### 특정 버전의 servable

이제 servable의 특정 버전을 지정하겠습니다. 하나만 있으므로 버전 1을 선택하겠습니다. 3가지 결과도 모두 살펴보겠습니다.

In [ ]:
# docs_infra: no_execute
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/fashion_model/versions/1:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

for i in range(0,3):
  show(i, 'The model thought this was a {} (class {}), and it was actually a {} (class {})'.format(
    class_names[np.argmax(predictions[i])], np.argmax(predictions[i]), class_names[test_labels[i]], test_labels[i]))